### Importações

In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time

In [2]:
# Caminhos para o dataset e dicionário
dataset_path = "Correto_whatsapp_rotulado_revisado.csv"
dic_path = "v2_SocialLIWC_formatado_ordenado.dic"

# Carrega o dataset
df = pd.read_csv(dataset_path)
df['text_content_anonymous'] = df['text_content_anonymous'].astype(str)

### Carrega o PrejudiceBR

In [3]:
# Função para carregar o dicionário
def carregar_dicionario_personalizado(dic_path):
    categorias = {}
    lexicon = {}
    dentro_das_categorias = False

    with open(dic_path, 'r', encoding='utf-8') as file:
        for linha in file:
            linha = linha.strip()
            
            # Detecta a seção de categorias delimitada por '%'
            if linha == '%':
                dentro_das_categorias = not dentro_das_categorias
                continue
            
            # Lê as categorias personalizadas
            if dentro_das_categorias:
                codigo, categoria = linha.split()
                categorias[codigo] = categoria
            else:
                # Lê as palavras e suas categorias
                partes = linha.split("\t")
                palavra = partes[0]
                categoria_ids = partes[1:]
                lexicon[palavra] = [categorias[codigo] for codigo in categoria_ids if codigo in categorias]
    
    return lexicon, list(categorias.values())

### Tokenizar e organizar as palavras como features

In [4]:
# Tokenização simples
def tokenize(text):
    tokens = []
    for match in re.finditer(r"\w+", text, re.UNICODE):
        tokens.append(match.group(0).lower())
    return tokens

# Função para incluir palavras do dicionário como colunas
def adicionar_palavras_como_features(df, lexicon):
    # Criar DataFrame temporário com todas as palavras inicializadas com 0
    new_columns = pd.DataFrame(0, index=df.index, columns=list(lexicon.keys()))
    
    # Concatenar com o DataFrame original de uma vez
    df = pd.concat([df, new_columns], axis=1)
    
    # Contar ocorrências de cada palavra no texto
    for i, texto in df['text_content_anonymous'].items():
        tokens = tokenize(texto)
        for token in tokens:
            if token in lexicon:
                df.at[i, token] += 1
                
    return df

### Configuração do baseline

In [5]:
# Carrega o dicionário
lexicon, category_names = carregar_dicionario_personalizado(dic_path)

# Adiciona palavras como features
df = adicionar_palavras_como_features(df, lexicon)

# Define as colunas de entrada (todas as palavras do dicionário) e o rótulo
X = df[list(lexicon.keys())]
y = df['preconceito']
total_features = X.shape[1] 

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

### Modelos

In [6]:
# Modelos
models = [
    LogisticRegression(), BernoulliNB(), MultinomialNB(), LinearSVC(dual=False),
    KNeighborsClassifier(), RandomForestClassifier(),
    GradientBoostingClassifier(n_estimators=200), MLPClassifier(batch_size=64, max_iter=50, early_stopping=True)
]

### Avaliação

In [7]:
# Avaliação de modelos com validação cruzada e teste final
resultados = []

for model in models:
    start_time = time.time()
    
    # Validação Cruzada (métricas médias dos folds de validação)
    metodos_scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    cv_results = cross_validate(model, X_train, y_train, cv=5, scoring=metodos_scoring, return_train_score=True)
    
    # Treina o modelo no conjunto de treino completo
    model.fit(X_train, y_train)
    
    # Prevê no conjunto de teste (holdout)
    y_pred = model.predict(X_test)
    
    # Métricas no teste
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred)
    
    # Calcula ROC AUC no teste (manuseia modelos sem predict_proba)
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_proba = model.decision_function(X_test)
    test_roc_auc = roc_auc_score(y_test, y_proba)
    
    end_time = time.time()
    
    # Compila resultados
    resultado = {
        'model': model.__class__.__name__,
        'execution_time': end_time - start_time,
        'total_features': total_features,
    }
    
    # Métricas de validação cruzada (média dos folds)
    for scoring in metodos_scoring:
        resultado[f'cv_{scoring}_avg'] = cv_results[f'test_{scoring}'].mean()
        resultado[f'cv_{scoring}_std'] = cv_results[f'test_{scoring}'].std()
    
    # Métricas no teste (holdout)
    resultado.update({
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1,
        'test_roc_auc': test_roc_auc
    })
    
    resultados.append(resultado)


In [8]:
# Exibe resultados formatados
df_resultados = pd.DataFrame(resultados)
cols_order = [
    'model', 'total_features', 'execution_time',
    'cv_accuracy_avg', 'test_accuracy',
    'cv_precision_avg', 'test_precision',
    'cv_recall_avg', 'test_recall',
    'cv_f1_avg', 'test_f1',
    'cv_roc_auc_avg', 'test_roc_auc'
]

print(df_resultados[cols_order].round(3))

                        model  total_features  execution_time  \
0          LogisticRegression             842           1.094   
1                 BernoulliNB             842           0.894   
2               MultinomialNB             842           0.587   
3                   LinearSVC             842           0.675   
4        KNeighborsClassifier             842           3.567   
5      RandomForestClassifier             842           4.685   
6  GradientBoostingClassifier             842           8.213   
7               MLPClassifier             842          15.034   

   cv_accuracy_avg  test_accuracy  cv_precision_avg  test_precision  \
0            0.828          0.828             0.930           0.934   
1            0.826          0.813             0.892           0.873   
2            0.827          0.815             0.884           0.871   
3            0.832          0.815             0.920           0.909   
4            0.770          0.797             0.840        

### Salva os resultados

In [9]:
# Salva as métricas no CSV
metrics_output_path = "wpp_dic_model_performance_metrics.csv"
df_resultados.to_csv(metrics_output_path, index=False)
print(f"Métricas salvas em: {metrics_output_path}")

# Salva o dataset processado
df.to_csv("wpp_processado.csv", index=False)

Métricas salvas em: wpp_dic_model_performance_metrics.csv
